In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import pandas as pd

# Load dataset
csv_path = "emoji_math_riddles.csv"
df = pd.read_csv(csv_path)
assert not df.empty, "❌ The dataset is empty"
print(f"✅ Successfully loaded dataset from {csv_path} ({len(df)} entries)")

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Formatting function
def format_data(example):
    return {
        "text": f"🧩 Puzzle: {example['problem']}\n Solution: {example['solution']}"
    }

dataset = dataset.map(format_data, remove_columns=["problem", "solution"])

# Train-test split
split_dataset = dataset.train_test_split(test_size=0.1)
print(f"🔹 Sample Training Data:\n{split_dataset['train'][0]['text']}")

# Model configuration
model_name = "Qwen/Qwen1.5-4B-Chat"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

# LoRA Configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_data(example):
    tokenized_output = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    labels = [(label if label != tokenizer.pad_token_id else -100) for label in tokenized_output["input_ids"]]
    tokenized_output["labels"] = labels
    return tokenized_output

tokenized_datasets = split_dataset.map(tokenize_data, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./emoji_math_model",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    fp16=torch.cuda.is_available(),
    save_total_limit=3,
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer
)

# Start training
trainer.train()

# Save trained model and tokenizer
model.save_pretrained("./emojis_math_model")
tokenizer.save_pretrained("./emojis_math_model")
print("✅ Model training is complete and model saved successfully!")


# 🔎 Testing model with emoji-based math problems
test_problems = [
    "🎵 + 🎵 = 14",
    "🏆 + 🏆 + 🏆 = 30",
    "🍎 + 🍎 + 🍎 + 🍎 = 20",
    "🚲 + 🚲 = 16",
    "🎮 + 🎮 + 🎮 = 27",
    "🏈 + 🏈 + 🏈 + 🏈 + 🏈 = 50",
    "🦉 + 🦉 = 22",
    "⌚ + ⌚ + ⌚ = 18",
    "🏡 + 🏡 + 🏡 + 🏡 = 32",
    "🎭 + 🎭 = 12"
]


device = "cuda" if torch.cuda.is_available() else "cpu"
print("\n🧠 AI is solving the riddles...")

for riddle in test_problems:
    input_text = f"🧩 Puzzle: {riddle}\n💡 Solution:"
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    output = model.generate(
        **inputs,
        max_length=60,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.6,  # Slightly different for more varied output
        top_p=0.85
    )

    print("-" * 50)
    print(f"❓ Riddle: {riddle}\n🤖 Answer: {tokenizer.decode(output[0], skip_special_tokens=True)}")



✅ Successfully loaded dataset from emoji_math_riddles.csv (30 entries)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

🔹 Sample Training Data:
🧩 Puzzle: 🌟 + 🌟 = 12
 Solution: 🌟 = 6


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 6,553,600 || all params: 3,956,922,880 || trainable%: 0.1656


Map:   0%|          | 0/27 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-f8087f1c7de9>:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,4.955955
2,No log,0.896326
3,No log,0.623733
4,3.782900,0.507518
5,3.782900,0.403869
6,3.782900,0.383777
7,3.782900,0.273991
8,0.379000,0.196713
9,0.379000,0.188628
10,0.379000,0.178072


✅ Model training is complete and model saved successfully!

🧠 AI is solving the riddles...
--------------------------------------------------
❓ Riddle: 🎵 + 🎵 = 14
🤖 Answer: 🧩 Puzzle: 🎵 + 🎵 = 14
💡 Solution: 🎵 = 7
--------------------------------------------------
❓ Riddle: 🏆 + 🏆 + 🏆 = 30
🤖 Answer: 🧩 Puzzle: 🏆 + 🏆 + 🏆 = 30
💡 Solution: 🏆 = 10
--------------------------------------------------
❓ Riddle: 🍎 + 🍎 + 🍎 + 🍎 = 20
🤖 Answer: 🧩 Puzzle: 🍎 + 🍎 + 🍎 + 🍎 = 20
💡 Solution:
--------------------------------------------------
❓ Riddle: 🚲 + 🚲 = 16
🤖 Answer: 🧩 Puzzle: 🚲 + 🚲 = 16
💡 Solution: 🚲 = 8
--------------------------------------------------
❓ Riddle: 🎮 + 🎮 + 🎮 = 27
🤖 Answer: 🧩 Puzzle: 🎮 + 🎮 + 🎮 = 27
💡 Solution: 🎮 = 9
--------------------------------------------------
❓ Riddle: 🏈 + 🏈 + 🏈 + 🏈 + 🏈 = 50
🤖 Answer: 🧩 Puzzle: 🏈 + 🏈 + 🏈 + 🏈 + 🏈 = 50
💡 Solution: 🏈 = 10
--------------------------------------------------
❓ Riddle: 🦉 + 🦉 = 22
🤖 Answer: 🧩 Puzzle: 🦉 + 🦉 = 22
💡 Solution: 🦉 = 10
---------

In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
!pip install torch transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 